In [80]:
import bibtexparser
import numpy as np

In [81]:
def getnames(names):
    """Make people names as firstnames surname. Should eventually combine up the two.

    :param names: a list of names
    :type names: list
    :returns: list -- Correctly formated names
    """
    tidynames = []
    for namestring in names:
        namestring = namestring.strip()
        if len(namestring) < 1:
            continue
        if ',' in namestring:
            namesplit = namestring.split(',', 1)
            last = namesplit[0].strip()
            firsts = [i.strip() for i in namesplit[1].split()]
        else:
            namesplit = namestring.split()
            last = namesplit.pop()
            firsts = [i.replace('.', '. ').strip() for i in namesplit]
        if last in ['jnr', 'jr', 'junior']:
            last = firsts.pop()
        for item in firsts:
            if item in ['ben', 'van', 'der', 'de', 'la', 'le']:
                last = firsts.pop() + ' ' + last
        tidynames.append(' '.join(firsts)+' '+last)
    return tidynames

In [82]:
from bibtexparser import customization

def customize(record):
    record = customization.convert_to_unicode(record)
    record = customization.author(record)
    return record

In [83]:
from bibtexparser.bparser import BibTexParser
with open('../../../../writing/CV/zhenwen.bib', 'r') as f:
    parser = BibTexParser()
    parser.customization = customize
    bib_db = bibtexparser.load(f, parser=parser)
    f.close()

In [84]:
year_list = [int(e['year']) for e in bib_db.entries]

In [85]:
bib_db.entries

[{u'%  pages': u'to appear',
  u'author': [u'Sheikh, Abdul-Saboor',
   u'\\mylastname, \\myfirstname',
   u'Harper, Nicol',
   u'Turner, Richard',
   u'L\xfccke, J\xf6rg'],
  u'booktitle': u'Computational and Systems Neuroscience (Cosyne)',
  'id': 'SheikhEtAl2015',
  u'title': u'Maximal causes for a masking based model of STRFs in primary auditory cortex',
  'type': u'inproceedings',
  u'year': u'2015'},
 {u'author': [u'Rodriguez-Serrano, Jose A.',
   u'Larlus, Diane',
   u'\\mylastname, \\myfirstname'],
  'id': 'Rodriguez-SerranoEtAl2014',
  u'journal': u'IEEE Transactions on Pattern Analysis and Machine Intelligence (TPAMI)',
  u'pages': u'under review',
  u'publisher': u'IEEE',
  u'title': u'Data-Driven Detection of Prominent Objects and of their Parts towards Improved Fine-Grained Visual Recognition',
  'type': u'article',
  u'year': u'2014'},
 {u'author': [u'\\mylastname, \\myfirstname',
   u'Damianou, Andreas',
   u'Hensman, James',
   u'Lawrence, Neil'],
  u'booktitle': u'NIPS 

In [93]:
order_list = np.argsort(year_list)[::-1]

In [88]:
def parse_bib_entry(entry):
    journaldetails = None
    if entry['type']=='inproceedings':
        pubname = entry['booktitle']
    elif entry['type']=='article':
        pubname = entry['journal']
        journaldetails = entry
    title = entry['title']
    authors = getnames(entry['author'])
    year = entry['year']
    #pages = entry['pages'] if 'pages' in entry else None
    return authors, title, pubname, year

In [111]:
def produce_publication_entry(entry):
    if entry['type'] != 'article' and entry['type']!='inproceedings':
        return ''
    authors, title, pubname,year = parse_bib_entry(entry)
    
    authors = [name if name!='\\myfirstname \\mylastname' else '*Zhenwen Dai*' for name in authors ]
    entry_str = ''
    entry_str += '+   '+', '.join(authors)+' ('+year+')  \n'
    entry_str += '    **'+title+'**  \n'
    entry_str += '    '+pubname+'  \n'
    entry_str += '\n'
    return entry_str

In [112]:
print produce_publication_entry(bib_db.entries[0])

+   Abdul-Saboor Sheikh, *Zhenwen Dai*, Nicol Harper, Richard Turner, Jörg Lücke (2015)  
    **Maximal causes for a masking based model of STRFs in primary auditory cortex**  
    Computational and Systems Neuroscience (Cosyne)  




In [113]:
def make_head():
    return """---
layout: page
title: My publications
tags: [about, Jekyll, theme, responsive]
comments: false
share: false
---
"""

In [114]:
#Produce publications.md
with open('./publications.md','w') as f:
    f.write(make_head().encode('utf8'))
    for i in order_list:
        f.write(produce_publication_entry(bib_db.entries[i]).encode('utf8'))
    f.close()    

In [115]:
print make_head()

---
layout: page
title: My publications
tags: [about, Jekyll, theme, responsive]
comments: false
share: false
---

